In [1]:
import numpy as np
import pandas as pd
import yaml
import pickle
from metrics import *

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
def load_config(config_path):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config
CATEGORIES_YAML = load_config("/hai/scratch/zwefers/seq2loc/metadata/level_classes.yaml")

In [6]:
trainset = pd.read_csv("/hai/scratch/zwefers/seq2loc/metadata/hpa_uniprot_combined_trainset.csv")
testset = pd.read_csv("/hai/scratch/zwefers/seq2loc/metadata/hou_testset.csv")
outdir = "outputs/seq2locbench/"
predictions_df = pd.DataFrame()

for model in ["prott5", "esm1"]:
    val_avg_df = []
    test_metrics_avg_df = []
    for level in [1,2,3]:
        val_perclass_df = []
        test_targets_all = []
        test_probs_all = []
        test_preds_all = []

        for fold in range(5):
            val_path = f"{outdir}/{model}/{fold}_1Layer_combined_level{level}_testout.pkl"
            val_df = pd.read_pickle(val_path)
            val_df = val_df.merge(trainset, 
                                left_on="ACC", 
                                right_on="uniprot_id", 
                                how="inner")
            val_probs = np.stack(val_df.preds.to_numpy())
            val_targets = []
            for locs in val_df[f"level{level}"].str.split(";").to_list():
                val_targets.append([1 if loc in locs else 0 
                                    for loc in CATEGORIES_YAML[f"level{level}"]])
            val_targets = np.array(val_targets)

            thresholds = [get_best_threshold_mcc(val_targets[:, i], val_probs[:, i]) 
                        for i in range(val_targets.shape[1])]
            thresholds = np.array(thresholds)
            #TODO: save thresholds

            _, val_metrics_perclass, val_metrics_avg = all_metrics(val_targets, 
                                                                val_probs, 
                                                                thresholds=thresholds)
            val_metrics_perclass["label"] = CATEGORIES_YAML[f"level{level}"]
            val_metrics_perclass["fold"] = fold
            val_metrics_avg["level"] = level
            val_metrics_avg["fold"] = fold
            val_perclass_df.append(val_metrics_perclass)
            val_avg_df.append(val_metrics_avg)

            test_path = f"{outdir}/{model}/{fold}_1Layer_combined_level{level}_hou_testout.pkl"
            test_df = pd.read_pickle(test_path)
            test_df = test_df.merge(testset, 
                                    left_on="ACC", 
                                    right_on="uniprot_id",
                                    how="inner")
            test_probs = np.stack(test_df.preds.to_numpy())
            test_targets = []
            for locs in test_df[f"level{level}"].str.split(";").to_list():
                test_targets.append([1 if loc in locs else 0 
                                    for loc in CATEGORIES_YAML[f"level{level}"]])
            test_targets = np.array(test_targets)


            test_preds = test_probs > thresholds[np.newaxis, :]
            test_max = (test_probs == test_probs.max(axis=1)[:, np.newaxis])
            test_preds = np.logical_or(test_preds, test_max)

            test_targets_all.append(test_targets)
            test_probs_all.append(test_probs)   
            test_preds_all.append(test_preds)
        
        val_perclass_df = pd.concat(val_perclass_df)
        val_perclass_df.to_csv(f"{outdir}/{model}/val_metrics_perclass_level{level}.csv", index=False)

        test_targets_all = np.array(test_targets_all)
        assert np.all(test_targets_all[0, :, :] == test_targets_all)
        test_targets = test_targets_all[0, :, :]
        test_probs = np.array(test_probs_all).mean(axis=0)
        #test_preds = np.array(test_preds_all).max(axis=0)
        test_preds = (np.array(test_preds_all).mean(axis=0) > 0.5).astype(np.int32)


        labels = np.array(CATEGORIES_YAML[f"level{level}"])
        predicted_labels = [set(labels[np.where(pred==1)[0]]) for pred in test_preds]
        predictions_df[f"DL2_{model}_level{level}_predictions"] = predicted_labels

        #Cut out empty categories in testset like int-fils and plastid
        idxs = np.where(test_targets.sum(axis=0) != 0)[0]
        test_targets = test_targets[:, idxs]
        test_probs = test_probs[:, idxs]
        test_preds = test_preds[:, idxs]
        thresholds = thresholds[idxs]

        _, test_metrics_perclass, test_metrics_avg = all_metrics(
                                                                test_targets, 
                                                                test_probs, 
                                                                y_pred_bin = test_preds,
                                                                thresholds=thresholds
                                                                )
        test_metrics_perclass["label"] = np.array(CATEGORIES_YAML[f"level{level}"])[idxs]
        test_metrics_perclass.to_csv(
            f"{outdir}/{model}/test_metrics_perclass_level{level}.csv", index=False)
        
        test_metrics_avg["level"] = level
        test_metrics_avg_df.append(test_metrics_avg)

    val_avg_df = pd.concat(val_avg_df)
    val_avg_df.to_csv(f"{outdir}/{model}/val_metrics_avg.csv", index=False)
    test_metrics_avg_df = pd.concat(test_metrics_avg_df)
    test_metrics_avg_df.to_csv(f"{outdir}/{model}/test_metrics_avg.csv", index=False)

In [7]:
testset.level1 = testset.level1.str.split(";").apply(lambda x: set(x))
testset.level2 = testset.level2.str.split(";").apply(lambda x: set(x))
testset.level3 = testset.level3.str.split(";").apply(lambda x: set(x))

In [8]:
pd.concat([testset, predictions_df], axis=1)

,uniprot_id,ensembl_id,level1,level2,level3,sequence,DL2_prott5_level1_predictions,DL2_prott5_level2_predictions,DL2_prott5_level3_predictions,DL2_esm1_level1_predictions,DL2_esm1_level2_predictions,DL2_esm1_level3_predictions
0,A0A087X0K9,ENSG00000104067,{plasma-membrane},{plasma-membrane},{plasma-membrane},MSARAAAAKSTAMEETAIWEQHTVTLHRAPGFGFGIAISGGRDNPH...,"{centrosome, vesicles, cytoskeleton, cytosol, ...","{vesicles, cytoskeleton, cytosol}","{cytoskeleton, cytosol}",{nucleoplasm},{nucleus},{nucleus}
1,A0A0C4DGS1,ENSG00000244038,{endoplasmic-reticulum},{endoplasmic-reticulum},{endomembrane-system},MEPSTAARAWALFWLLLPLLGAVCASGPRTLVLLDNLNVRETHSLF...,"{centrosome, vesicles, cytoskeleton, cytosol, ...","{cytoskeleton, cytosol}","{cytoskeleton, cytosol}","{nuclear-bodies, nucleoplasm, nucleoli-fibrill...",{nucleus},{nucleus}
2,A0A1B0GTU4,ENSG00000089159,{plasma-membrane},{plasma-membrane},{plasma-membrane},MDDLDALLADLESTTSHISKRPVFLSEETPYSYPTGNHTYQEIAVP...,"{vesicles, nucleoplasm, microtubules}","{vesicles, nucleoli, nucleus}","{nucleoli, nucleus}","{nuclear-bodies, nucleoplasm, nucleoli-fibrill...",{nucleus},{nucleus}
3,A0A2R8YG42,ENSG00000102606,{plasma-membrane},{plasma-membrane},{plasma-membrane},MNSAEQTVTWLITLGVLESPKKTISDPEGFLQASLKDGVVLCRLLE...,"{nuclear-bodies, vesicles, cytosol, nucleoplas...","{vesicles, cytosol, nucleus}",{nucleus},"{lysosomes, golgi-apparatus, vesicles, interme...","{vesicles, plasma-membrane}","{endomembrane-system, plasma-membrane}"
4,A0A6Q8PGB0,ENSG00000181222,{nucleoplasm},{nucleus},{nucleus},MHGGGPPSGDSACPLRTIKRVQFGVLSPDELKRMSVTEGGIKYPET...,"{nuclear-bodies, centrosome, vesicles, interme...","{vesicles, cytoskeleton, cytosol}","{cytoskeleton, cytosol}","{nuclear-bodies, intermediate-filaments, cytos...","{nucleoli, nucleus}",{nucleus}
...,...,...,...,...,...,...,...,...,...,...,...,...
3809,Q9Y6X4,ENSG00000198780,{nuclear-membrane},{nucleus},{nucleus},MAFPVDMLENCSHEELENSAEDYMSDLRCGDPENPECFSLLNITIP...,"{nucleoplasm, cytosol}","{cytosol, nucleus}",{cytosol},"{nucleoplasm, cytosol}","{cytosol, nucleus}","{cytosol, nucleus}"
3810,Q9Y6X8,ENSG00000178764,{nucleoplasm},{nucleus},{nucleus},MASKRKSTTPCMVRTSQVVEQDVPEEVDRAKEKGIGTPQPDVAKDS...,"{nucleoplasm, nucleoli-fibrillar-center, nucle...","{nucleoli, nucleus}","{nucleoli, nucleus}","{nucleoplasm, nucleoli}","{cytosol, nucleus}","{cytosol, nucleus}"
3811,Q9Y6X9,ENSG00000133422,"{nucleoplasm, cytosol}","{cytosol, nucleus}","{cytosol, nucleus}",MAFTNYSSLNRAQLTFEYLHTNSTTHEFLFGALAELVDNARDADAT...,"{vesicles, mitochondria, endoplasmic-reticulum...","{vesicles, mitochondria}",{mitochondria},"{mitochondria, endoplasmic-reticulum}",{endoplasmic-reticulum},{mitochondria}
3812,Q9Y6Y0,ENSG00000116679,{cytosol},{cytosol},{cytosol},MIPNGYLMFEDENFIESSVAKLNALRKSGQFCDVRLQVCGHEMLAH...,{mitochondria},{mitochondria},{mitochondria},"{vesicles, mitochondria, endoplasmic-reticulum}",{mitochondria},{mitochondria}


In [9]:
test_metrics_avg_df

,macro_ap,micro_ap,acc,f1_macro,f1_micro,jaccard_macro,jaccard_micro,rocauc_macro,rocauc_micro,mlrap,cov_error,num_labels,level
0,0.324091,0.422995,0.099371,0.335974,0.403581,0.230035,0.403581,0.845203,0.874091,0.637396,4.271106,3.659413,1
0,0.589578,0.671429,0.337703,0.578397,0.633956,0.424029,0.633956,0.891232,0.904055,0.786128,2.188778,1.753277,2
0,0.661898,0.703067,0.440482,0.639124,0.685319,0.485208,0.685319,0.892814,0.897390,0.810943,1.974043,1.505506,3
